In [1]:
import numpy as np
import pandas as pd

In [2]:
sigmoid = lambda x: 1 / (1 + np.exp(-x))

In [198]:
# TODO: Alterar neurômios para iniciarem sem entrada e com seus pesos baseados em topology
# TODO: Função update - alterar o valor dos x e passar adiante (u e g)
class Neuron:
  def __init__(self, x=None, w=[], function=sigmoid):
    # function - Função de ativação
    # x - Entrada
    # w - Peso
    # u - Saída do corpo
    # g - Saída da função de ativação
    # pd - Derivada parcial
    
    self.function = function
    self.x = x
    self.pd = None
    
    if self.is_bias():
      self.w = w
      self.u = None
      self.g = x
    else:
      self.w = np.random.random(len(self.x)) if w == [] else w
      self.u = self.x @ self.w
      self.g = self.function(self.u)
      
  def update(self, new_x):
    if not self.is_bias():
      self.x = new_x
      self.u = self.x @ self.w
      self.g = self.function(self.u)
  
  is_bias = lambda self: True if type(self.x) is not list else False
  
  __repr__ = lambda self: '{} @ {} ({}) -> {}'.format(self.x, self.w, self.pd, self.g)

In [190]:
class MLP:
  def __init__(self, topology):
    self.inputs = inputs
    self.desired = desired
    self.topology = topology
    self.precision = precision
    self.learning_rate = learning_rate
    self.errors = [0] * self.topology[-1]
    self.total_error = 0
    
#     self.init_neurons()
    self.inputs = [.05, .1]
    self.desired = [.01, .99]
    self.learning_rate = 0.5
    self.layers = [[Neuron(.05), Neuron(.1), Neuron(1)]]
    self.layers += [[Neuron([e.g for e in self.layers[0]], np.array([.15, .2, .35])), 
                     Neuron([e.g for e in self.layers[0]], np.array([.25, .3, .35])), 
                     Neuron(1)]]
    self.layers += [[Neuron([e.g for e in self.layers[1]], np.array([.4, .45, .6])), 
                     Neuron([e.g for e in self.layers[1]], np.array([.5, .55, .6]))]]
    
    
  def init_neurons(self):
    # Camada de entrada
    self.layers = [[Neuron(self.inputs[i]) for i in range(self.topology[0])]]
    self.layers[0] += [Neuron(1)] # Bias 
    
    # Camadas ocultas
    for i in range(1, len(self.topology)):
      self.layers += [[Neuron([e.g for e in self.layers[i - 1]]) for j in range(self.topology[i])]]
      if i != len(self.topology) - 1: self.layers[i] += [Neuron(1)] # Bias

  def update_neurons(self):
    for i in range(1, len(self.layers)):
      [n.update([e.g for e in self.layers[i - 1]]) for n in self.layers[i]]
        
  def update_weights(self):
    # Camada de saída
    for i in range(len(self.layers[-1])):
      # Derivada parcial
      neuron = self.layers[-1][i]
      neuron.pd = - (self.desired[i] - neuron.g) * neuron.g * (1 - neuron.g)
      # Pesos
      for j in range(len(neuron.w)):
        neuron.w[j] -= self.learning_rate * neuron.pd * self.layers[-2][j].g
          
    # Camadas ocultas
    for i in range(len(self.layers) - 2, 0, -1):
      for j in range(len(self.layers[i])):
        # Derivada parcial
        neuron = self.layers[i][j]
        descendents = sum([n.pd * n.w[j] for n in self.layers[i + 1] if not n.is_bias()])
        neuron.pd = descendents * neuron.g * (1 - neuron.g)
        # Pesos
        for k in range(len(neuron.w)):
          neuron.w[k] -= self.learning_rate * neuron.pd * neuron.x[k]
  
  def update_errors(self):
    out_layer = self.layers[-1]
    out_length = len(out_layer)
    
    self.errors = [(out_layer[i].g - self.desired[i]) ** 2 for i in range(out_length)]
    self.total_error = sum(self.errors) / 2
    
  def train(self):
  
  def __repr__(self):
    result = 'Entradas: {}\nSaída desejada: {}\nErro total: {}\n\n'.format(self.inputs, self.desired, self.total_error)
    result += 'Camada 0 - Entrada\n{}\n\n'.format(self.layers[0])
    for i in range(1, len(self.layers) - 1): result += 'Camada {} - Escondida \n{}\n\n'.format(i, self.layers[i])
    result += 'Camada {} - Saída\n{}\n\n'.format(len(self.layers) - 1, self.layers[-1])
    
    return result

In [195]:
MLP([1, 0],[1],[2, 3, 1], 0.1)

/home/impsid/.anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


Entradas: [0.05, 0.1]
Saída desejada: [0.01, 0.99]
Erro total: 2.4381048904383838e-06

Camada 0 - Entrada
[0.05 @ [] (None) -> 0.05, 0.1 @ [] (None) -> 0.1, 1 @ [] (None) -> 1]

Camada 1 - Escondida 
[[0.05, 0.1, 1] @ [0.18291248 0.26582495 1.0082495 ] (-1.0213877377663664e-05) -> 0.7396167616057072, [0.05, 0.1, 1] @ [0.28218874 0.36437748 0.99377485] (-1.0221184044840948e-05) -> 0.7396830843499616, 1 @ [] (-0.0) -> 1]

Camada 2 - Saída
[[0.7396167616057072, 0.7396830843499616, 1] @ [-1.45152738 -1.40484546 -2.33377952] (1.8137503158944566e-05) -> 0.01158382711768914, [0.7396167616057072, 0.7396830843499616, 1] @ [1.52214253 1.57305739 2.16111789] (-1.7546845346810148e-05) -> 0.9884617683899352]
